In [2]:
import sys
import os
from pathlib import Path
path_project = Path(os.getcwd()).parent # /home/o.lipina/workspace/rucubrik
path_src = path_project / 'src'  # /home/o.lipina/workspace/rucubrik/src
sys.path.append(str(path_src))
sys.path.append(str(path_project))

%load_ext autoreload
%autoreload 2

In [3]:
import requests
import json
import logging
import json
# import pandas as pd
import pickle
from pathlib import Path
import re
import random
import time
from tqdm import tqdm
from urllib.parse import urlparse
import warnings
warnings.simplefilter("ignore")
# import numpy as np

random.seed(42)
# np.random.seed(42)
import datetime

In [4]:
from src.config import CONFIG

In [5]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(levelname)s] %(name)s - %(message)s"
)

## Click tool

In [5]:
from src.tools.browser_2.environment import BrowserEnvironment
from src.tools.browser_2.click import ClickTool
from src.tools.browser_2.click import FillTool
from src.tools.browser_2.click import CheckContentTool
from src.tools.browser_2.dropdown import DropDownTool
from src.tools.browser_2.dropdown import DropdownOptionsTool


[INFO] config - Configuration loaded successfully from /home/olipina/workspace/org-agent/config.yaml


In [6]:
CONFIG.isu_booking_creds.booking_login

'https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688'

In [7]:
browser_env = BrowserEnvironment()
await browser_env.initialize()

[INFO] src.tools.browser_2.environment - Starting new browser environment


### Login

In [8]:
await browser_env.navigate(CONFIG.isu_booking_creds.booking_login)

In [9]:
browser_env.current_url

'https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688'

In [10]:
result = await FillTool().execute(
        env=browser_env,
        selector='#username',
        value=CONFIG.isu_booking_creds.username
        )
result = await FillTool().execute(
        env=browser_env,
        selector='#password',
        value=CONFIG.isu_booking_creds.password
        )

[INFO] src.tools.browser_2.click - Attempting to fill '311408' into input #username
[INFO] src.tools.browser_2.click - Successfully filled '311408' into #username
[INFO] src.tools.browser_2.click - Attempting to fill 'musummer1' into input #password


[INFO] src.tools.browser_2.click - Successfully filled 'musummer1' into #password


In [11]:
result = await ClickTool().execute(
        env=browser_env,
        text="Вход",
        
    )

[INFO] src.tools.browser_2.click - Attempting to click button with text: Вход


[INFO] src.tools.browser_2.click - Waiting for navigation to complete
[ERROR] src.tools.browser_2.click - Click failed on text='Вход': 'neurl_after_clickw_url'


In [12]:
texts=["Личный кабинет", "Центр приложений"]
login_result = await CheckContentTool().execute(
    env=browser_env,texts=["Личный кабинет", "Центр приложений"])

[INFO] src.tools.browser_2.click - Found texts: ['Личный кабинет', 'Центр приложений']


### Go to booking

In [54]:
await browser_env.navigate(CONFIG.isu_booking_creds.booking_url)

In [55]:
browser_env.current_url

'https://isu.ifmo.ru/pls/apex/f?p=2431:1:117559366568926:'

In [56]:
result = await ClickTool().execute(
        env=browser_env,
        text="Создать заявку",
    )

[INFO] src.tools.browser_2.click - Attempting to click button with text: Создать заявку


[INFO] src.tools.browser_2.click - Waiting for navigation to complete
[INFO] src.tools.browser_2.click - Click successful on text='Создать заявку'. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


In [22]:
expected_texts = [
                "Карточка заявки",
                "Заявка",
                "Картотека заявок",
                "Проект заявки"
            ]
result = await CheckContentTool().execute(
    env=browser_env,texts=expected_texts)


[INFO] src.tools.browser_2.click - Found texts: ['Карточка заявки', 'Заявка', 'Картотека заявок', 'Проект заявки']


### Specify building

In [57]:
result = await DropdownOptionsTool().execute(
    env=browser_env,
    dropdown_selector = '.select2-chosen',
)

[INFO] src.tools.browser_2.dropdown - Opening dropdown by selector: .select2-chosen


[INFO] src.tools.browser_2.dropdown - Retrieving dropdown options
[INFO] src.tools.browser_2.dropdown - Closing dropdown with Escape key
[INFO] src.tools.browser_2.dropdown - Successfully retrieved 21 options


In [24]:
result.meta['options'].split(', ')

['Коворкинги и пространства',
 'ITMO PLACE (Ломоносова 9)',
 'Kronbars Space (Ломоносова 9)',
 'Коворкинг 1300 (Кронверкский 49)',
 'Тихий коворкинг в аудитории 1302',
 'Коворкинг Ломоносова 1303 (Ломоносова 9)',
 'Коворкинг Студенческого офиса 1400 (Ломоносова 9)',
 'Музей истории Университета ИТМО (Гривцова 14-16)',
 'Портал в Яндекс (Кронверкский 49)',
 'Общий аудиторный фонд',
 'Аудиторный фонд ИТМО (Гривцова 14-16)',
 'Аудиторный фонд ИТМО (Кронверкский 49)',
 'Аудиторный фонд ИТМО (Ломоносова 9)',
 'Аудиторный фонд ИТМО (Чайковского 11/2)',
 'Актовые залы',
 'Актовый зал (Гривцова 14-16)',
 'Актовый зал (Ломоносова 9)',
 'Конференц-залы',
 'Овальный зал пер (Гривцова 14-16)',
 'Виртуальные аудитории',
 'Виртуальные аудитории zoom']

In [25]:
expected_building = CONFIG.default_booking_params.building
expected_building

'Аудиторный фонд ИТМО (Кронверкский 49)'

In [58]:
result1 = await DropDownTool().execute(
        env=browser_env,
        option_text=expected_building,
        dropdown_selector='.select2-chosen',
    )

[INFO] src.tools.browser_2.dropdown - Clicking dropdown by selector: .select2-chosen
[INFO] src.tools.browser_2.dropdown - Selecting option: Аудиторный фонд ИТМО (Кронверкский 49)


[INFO] src.tools.browser_2.dropdown - Successfully selected 'Аудиторный фонд ИТМО (Кронверкский 49)' from dropdown


In [59]:
expected_texts = [expected_building]
result = await CheckContentTool().execute(
    env=browser_env,texts=expected_texts)


[INFO] src.tools.browser_2.click - Found texts: ['Аудиторный фонд ИТМО (Кронверкский 49)']


### Select date

In [60]:
res = ClickTool()
click_result = await ClickTool().execute(
            env=browser_env,
            selector="#P4_DATE",
            wait_for_navigation=False,
)

[INFO] src.tools.browser_2.click - Attempting to click element by selector: #P4_DATE
[INFO] src.tools.browser_2.click - Click successful on #P4_DATE. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


In [61]:
calendar_check = await CheckContentTool().execute(
                env=browser_env,
                texts=["ui-datepicker-calendar", "ui-datepicker-month", "ui-datepicker-year"]
            )

[INFO] src.tools.browser_2.click - Found texts: ['ui-datepicker-calendar', 'ui-datepicker-month', 'ui-datepicker-year']


In [62]:
from datetime import datetime
date_str = "18-01-2025"
target_date = datetime.strptime(date_str, "%d-%m-%Y")

In [63]:
date_selector = (
                f'[data-handler="selectDay"]'
                f'[data-month="{target_date.month - 1}"]'  # jQuery UI months are 0-based
                f'[data-year="{target_date.year}"]'
                f' a:text("{target_date.day}")'
            )
            
date_click = await ClickTool().execute(
                env=browser_env,
                selector=date_selector,
                wait_for_navigation=False,
            )

[INFO] src.tools.browser_2.click - Attempting to click element by selector: [data-handler="selectDay"][data-month="0"][data-year="2025"] a:text("18")
[INFO] src.tools.browser_2.click - Click successful on [data-handler="selectDay"][data-month="0"][data-year="2025"] a:text("18"). URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


In [64]:
input_value = await browser_env.page.input_value("#P4_DATE")
input_value

'18.01.2025'

In [65]:
target_date_dots_str = datetime.strptime(date_str, "%d-%m-%Y").strftime("%d.%m.%Y")
target_date_dots_str

'18.01.2025'

In [66]:
calendar_check = await CheckContentTool().execute(
                env=browser_env,
                texts=[target_date_dots_str]
            )

[INFO] src.tools.browser_2.click - Found texts: ['18.01.2025']


## Бронь аудиторий

### Выбор клеточки в расписании

In [35]:
click_tool = await ClickTool().execute(
                env=browser_env,
                selector="#P4_AUD_NAME",
                wait_for_navigation=False,
            )


[INFO] src.tools.browser_2.click - Attempting to click element by selector: #P4_AUD_NAME
[INFO] src.tools.browser_2.click - Click successful on #P4_AUD_NAME. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


In [36]:
# free room
roomid="29"
interval="3"
# booked room
cell_selector = (
                f'td.reserve[roomid="{roomid}"][interval="{interval}"]'
            )
cell = await browser_env.page.query_selector(cell_selector)

if not cell:
    error=f"No available slot found for room {roomid} at interval {interval}"
cell

<JSHandle preview=JSHandle@<td roomid="29" interval="3" class="reserve">…</td>>

In [37]:
cell_click = await ClickTool().execute(
                env=browser_env,
                selector=cell_selector,
                wait_for_navigation=False,
            )

[INFO] src.tools.browser_2.click - Attempting to click element by selector: td.reserve[roomid="29"][interval="3"]


[INFO] src.tools.browser_2.click - Click successful on td.reserve[roomid="29"][interval="3"]. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


In [38]:
texts = [
    "Требуется техническое сопровождение мероприятия",
    "Контактный телефон",
    "Название мероприятия"
]
check = await CheckContentTool().execute(
                env=browser_env,
                texts=texts
            )
check

[INFO] src.tools.browser_2.click - Found texts: ['Требуется техническое сопровождение мероприятия', 'Контактный телефон', 'Название мероприятия']


BrowserToolResponse(success=True, error=None, meta={'action': 'check_content', 'url': 'https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,', 'texts': 'Требуется техническое сопровождение мероприятия, Контактный телефон, Название мероприятия', 'narrative': "Found texts: ['Требуется техническое сопровождение мероприятия', 'Контактный телефон', 'Название мероприятия']", 'found_texts': 'Требуется техническое сопровождение мероприятия, Контактный телефон, Название мероприятия', 'missing_texts': ''})

In [39]:
datetime.strptime(
            CONFIG.default_booking_params.start_time, "%H:%M").time()

datetime.time(19, 0)

### Определение нужной аудитории

In [52]:
from src.scenarios.booking import BookingParams
from src.tools.booking_utils import get_intervals_for_timespan, get_room_id_from_table


In [89]:
booking_params = BookingParams(room_count=3)

[autoreload of src.scenarios.booking failed: Traceback (most recent call last):
  File "/home/olipina/workspace/org-agent/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/olipina/workspace/org-agent/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/olipina/workspace/org-agent/src/scenarios/booking.py", line 29, in <module>
    class BookingParams(BaseModel):
  File "/home/olipina/workspace/org-agent/src/scenarios/booking.py", line 47, in BookingParams
    default=default_booking_params.room

In [48]:
booking_params

BookingParams(room_count=3, date=datetime.datetime(2025, 1, 16, 15, 19, 28, 440663), start_time=datetime.time(19, 0), end_time=datetime.time(22, 50), building='Аудиторный фонд ИТМО (Кронверкский 49)')

In [49]:
start_time = booking_params.start_time
end_time = booking_params.end_time

In [81]:
from src.tools.booking_utils import GetIntervalsForTimespanTool


time_intervals = GetIntervalsForTimespanTool().execute(start_time, end_time)
time_intervals

[INFO] src.tools.booking_utils - Time 19:00:00-22:50:00 overlaps with interval 7 (18:40:00-20:10:00)
[INFO] src.tools.booking_utils - Time 19:00:00-22:50:00 overlaps with interval 8 (20:20:00-21:50:00)
[INFO] src.tools.booking_utils - Time 19:00:00-22:50:00 overlaps with interval 9 (22:00:00-23:00:00)


['7', '8', '9']

In [68]:
page_content =  await browser_env.page.content()

In [75]:
from src.tools.booking_utils import GetRoomIdFromTableTool


GetRoomIdFromTableTool().execute(
    page_content = await browser_env.page.content(),
    room_number = "2403"
)

[INFO] src.tools.booking_utils - Found room_id 59 for room number 2403


'59'

In [82]:
schedule_res = await ClickTool().execute(
                env=browser_env,
                selector="#P4_AUD_NAME",
                wait_for_navigation=False,
            )


[INFO] src.tools.browser_2.click - Attempting to click element by selector: #P4_AUD_NAME
[INFO] src.tools.browser_2.click - Click successful on #P4_AUD_NAME. URL after click: https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,


### А если она забронирована?

In [84]:
await CheckContentTool().execute(
                env=browser_env,
                texts=["scheduleTable"]
            )

[INFO] src.tools.browser_2.click - Found texts: ['scheduleTable']


BrowserToolResponse(success=True, error=None, meta={'action': 'check_content', 'url': 'https://isu.ifmo.ru/pls/apex/f?p=2431:4:117559366568926::NO:4:P4_MIN_DATE,P4_BACK_PAGE:,', 'texts': 'scheduleTable', 'narrative': "Found texts: ['scheduleTable']", 'found_texts': 'scheduleTable', 'missing_texts': ''})

In [ ]:
initial_room_number = "2403"
interval_human_readable = UNIVERSITY_INTERVALS[interval]
...
room_id = "59"
interval = "2"
# interval = "7"
cell_selector = (
                f'td.reserve[roomid="{room_id}"][interval="{interval}"]'
            )
cell = await browser_env.page.query_selector(cell_selector)
logger.info("Success – cell found. Room %s is free at interval %s", initial_room_number, interval_human_readable)
if cell is None:
    cell_selector = (
                f'td.busy[roomid="{room_id}"]'
            )
    cell = await browser_env.page.query_selector(cell_selector)
    if cell is None:
        logger.info("No cell found for room %s,roomid=%s probably doesn't exist.", initial_room_number, room_id)
    else:
        logger.info("Cell found. It means that room %s, roomid=%s is already booked at interval %s", initial_room_number, room_id, interval)

cell 2 JSHandle@node


# ScenarioSteps

In [7]:
from src.scenarios.booking_steps import LoginStep
from src.tools.browser.environment import BrowserEnvironment

In [16]:
browser_env = BrowserEnvironment()
await browser_env.initialize()

[INFO] src.tools.browser.environment - Starting new browser environment


In [20]:
login_step = LoginStep()

In [21]:
await login_step.execute(browser_env)

[INFO] src.scenarios.booking_steps - Starting login step execution
[INFO] src.scenarios.booking_steps - Navigating to login page: https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688
[INFO] src.tools.browser.click - Attempting to navigate to https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688
[INFO] src.tools.browser.click - Navigation successful to https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688
[INFO] src.tools.browser.click - Found texts: ['Имя пользователя или E-mail']
[INFO] src.scenarios.booking_steps - Filling use

True

In [26]:
login_step.execution_history[0].response

ToolResponse(success=True, error=None, meta={'action': 'navigate', 'url': 'https://id.itmo.ru/auth/realms/itmo/protocol/openid-connect/auth?response_type=code&scope=openid&client_id=isu&redirect_uri=https://isu.ifmo.ru/api/sso/v1/public/login?apex_params=p=2143:LOGIN:112820233588688'})